# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

This project is to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd 
import xlsxwriter
from scipy import stats
import requests
import math

## Importing List of Stocks & API Token

In [2]:
stocks  = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making The First API Call
We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [6]:
data

{'avgTotalVolume': 69227132,
 'calculationPrice': 'tops',
 'change': -2.06,
 'changePercent': -0.01228,
 'close': 0,
 'closeSource': 'fiialcof',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 174.85,
 'iexAskSize': 115,
 'iexBidPrice': 172.82,
 'iexBidSize': 225,
 'iexClose': 172.15,
 'iexCloseTime': 1725444119769,
 'iexLastUpdated': 1722599904229,
 'iexMarketPercent': 0.019946004141820517,
 'iexOpen': 176.184,
 'iexOpenTime': 1677779481555,
 'iexRealtimePrice': 177.78,
 'iexRealtimeSize': 103,
 'iexVolume': 939998,
 'lastTradeTime': 1700444003323,
 'latestPrice': 174.13,
 'latestSource': 'IEX real time price',
 'latestTime': '2:03:22 PM',
 'latestUpdate': 1695688295103,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,

## Parsing The API Call
This API call has the metric we need - the price-to-earnings ratio.

In [7]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

29.12

## Executing A Batch API Call & Building Our DataFrame

It's now time to execute several batch API calls and add the information we need to our DataFrame.

It contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url =f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,145.030,34,N/A
1,AAL,14.090,-4.98,N/A
2,AAP,212.057,23.03,N/A
3,AAPL,176.760,29.06,N/A
4,ABBV,149.630,21.19,N/A
...,...,...,...,...
496,YUM,117.270,24.04,N/A
497,ZBH,113.290,107.97,N/A
498,ZBRA,345.770,35.51,N/A
499,ZION,57.990,6.53,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [10]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [11]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,43.854,3.13,N/A
1,AIG,57.860,3.85,N/A
2,NUE,142.200,4.38,N/A
3,APA,37.820,4.42,N/A
4,KSS,31.190,4.59,N/A
5,DHI,74.810,4.89,N/A
6,PVH,69.270,4.93,N/A
7,PHM,44.490,4.96,N/A
8,PFG,80.340,4.99,N/A
9,COF,115.410,5,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

In [12]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [13]:
portfolio_input()

Enter the value of your portfolio:1000000


Now using the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [14]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,43.854,3.13,456
1,AIG,57.860,3.85,345
2,NUE,142.200,4.38,140
3,APA,37.820,4.42,528
4,KSS,31.190,4.59,641
5,DHI,74.810,4.89,267
6,PVH,69.270,4.93,288
7,PHM,44.490,4.96,449
8,PFG,80.340,4.99,248
9,COF,115.410,5,173


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [15]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [16]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,189.500,N/A,62.12,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,312.925,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,301.390,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,98.120,N/A,50.17,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.13,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.750,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,51.610,N/A,14.58,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,59.220,N/A,33.4,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,34.410,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,110.790,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

We will replace missing data with the average non-`NaN` data point from that column. 

In [18]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [19]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [20]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.746507
1      0.033932
2       0.52495
3      0.660679
4       0.47505
         ...   
496    0.540918
497     0.98004
498    0.768463
499    0.095808
500    0.802395
Name: PE Percentile, Length: 501, dtype: object
0      0.768463
1      0.061876
2      0.600798
3      0.978044
4       0.93014
         ...   
496    0.057884
497    0.295409
498    0.754491
499    0.222555
500    0.932136
Name: PB Percentile, Length: 501, dtype: object
0      0.828343
1      0.011976
2      0.177645
3      0.848303
4      0.716567
         ...   
496    0.772455
497    0.560878
498    0.541916
499    0.452096
500     0.92016
Name: PS Percentile, Length: 501, dtype: object
0      0.824351
1      0.996008
2      0.419162
3      0.790419
4      0.331337
         ...   
496    0.728543
497    0.692615
498    0.688623
499    0.097804
500    0.894212
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0      0.802395
1       0.02994
2      0.163673
3      0.938124
4      0.546906
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.200,N/A,33.64,0.746507,7.89,0.768463,6.3700,0.828343,23.474397,0.824351,11.865240,0.802395,N/A
1,AAL,14.125,N/A,-5.05,0.033932,-1.20,0.061876,0.2585,0.011976,112.622239,0.996008,0.849035,0.02994,N/A
2,AAP,213.630,N/A,22.90,0.52495,4.52,0.600798,1.1700,0.177645,12.987539,0.419162,2.733320,0.163673,N/A
3,AAPL,175.360,N/A,28.79,0.660679,48.24,0.978044,7.3500,0.848303,22.576881,0.790419,17.475647,0.938124,N/A
4,ABBV,148.340,N/A,20.81,0.47505,17.60,0.93014,4.5000,0.716567,11.137005,0.331337,7.579653,0.546906,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.970,N/A,23.62,0.540918,-4.04,0.057884,5.1700,0.772455,20.023241,0.728543,9.334130,0.690619,N/A
497,ZBH,115.000,N/A,109.04,0.98004,2.05,0.295409,3.3500,0.560878,18.386922,0.692615,5.900689,0.413174,N/A
498,ZBRA,345.750,N/A,36.34,0.768463,7.43,0.754491,3.2100,0.541916,18.348392,0.688623,7.416213,0.526946,N/A
499,ZION,59.830,N/A,6.69,0.095808,1.74,0.222555,2.5800,0.452096,5.294374,0.097804,2.484597,0.145709,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [21]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.200,N/A,33.64,0.746507,7.89,0.768463,6.3700,0.828343,23.474397,0.824351,11.865240,0.802395,0.794012
1,AAL,14.125,N/A,-5.05,0.033932,-1.20,0.061876,0.2585,0.011976,112.622239,0.996008,0.849035,0.02994,0.226747
2,AAP,213.630,N/A,22.90,0.52495,4.52,0.600798,1.1700,0.177645,12.987539,0.419162,2.733320,0.163673,0.377246
3,AAPL,175.360,N/A,28.79,0.660679,48.24,0.978044,7.3500,0.848303,22.576881,0.790419,17.475647,0.938124,0.843114
4,ABBV,148.340,N/A,20.81,0.47505,17.60,0.93014,4.5000,0.716567,11.137005,0.331337,7.579653,0.546906,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.970,N/A,23.62,0.540918,-4.04,0.057884,5.1700,0.772455,20.023241,0.728543,9.334130,0.690619,0.558084
497,ZBH,115.000,N/A,109.04,0.98004,2.05,0.295409,3.3500,0.560878,18.386922,0.692615,5.900689,0.413174,0.588423
498,ZBRA,345.750,N/A,36.34,0.768463,7.43,0.754491,3.2100,0.541916,18.348392,0.688623,7.416213,0.526946,0.656088
499,ZION,59.830,N/A,6.69,0.095808,1.74,0.222555,2.5800,0.452096,5.294374,0.097804,2.484597,0.145709,0.202794


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [22]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [23]:
portfolio_input()

Enter the value of your portfolio:1000000


In [24]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\lenovo\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,30.870,647,4.65,0.055888,1.0400,0.103792,0.2420,0.007984,3.645405,0.03992,1.104920,0.0499,0.051497
1,AIG,57.420,348,3.81,0.0499,0.9820,0.095808,0.7690,0.103792,2.203948,0.017964,0.754197,0.017964,0.057086
2,UNM,39.900,501,7.25,0.113772,0.8304,0.075848,0.6648,0.077844,4.092329,0.05988,0.664529,0.011976,0.067864
3,PVH,70.100,285,4.86,0.05988,0.9323,0.085828,0.5219,0.051896,5.251201,0.095808,1.160598,0.055888,0.06986
4,GM,39.980,500,7.63,0.122754,0.9315,0.083832,0.4473,0.03992,2.334865,0.01996,1.836965,0.0998,0.073253
5,GPS,10.071,1985,-54.13,0.00998,1.7400,0.222555,0.2670,0.013972,5.985369,0.125749,0.836843,0.027944,0.08004
6,CAH,70.960,281,-20.79,0.01996,-27.7500,0.027944,0.1085,0.001996,7.449381,0.179641,2.960378,0.183633,0.082635
7,SYF,34.703,576,5.00,0.063872,1.3900,0.144711,1.0339,0.145709,3.217911,0.027944,1.089467,0.047904,0.086028
8,COF,113.555,176,5.08,0.065868,0.8397,0.077844,1.3000,0.206587,3.574763,0.033932,1.365616,0.06986,0.090818
9,F,16.145,1238,5.59,0.073852,1.4900,0.164671,0.4454,0.037924,4.623217,0.073852,2.348628,0.133733,0.096806


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

In [25]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

In [28]:
writer.save()